# **Group 16: airbnb in European cities - Price Prediction**

In [4]:
#import tensorflow as tf
from IPython.display import display, Image
# from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn import tree
import pickle
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from scipy.stats import randint
import graphviz
import xgboost as xgb
from xgboost import XGBRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import keras_tuner as kt
import tensorflow as tf


ImportError: cannot import name 'type_spec_registry' from 'tensorflow.python.framework' (/opt/anaconda3/envs/python_3_10/lib/python3.10/site-packages/tensorflow/python/framework/__init__.py)

## **1. Data loading and merging**

**Dataset Introduction**

This dataset offers a comprehensive examination of airbnb prices in popular European cities. 

It covers a range of attributes for each listing, including room types, cleanliness and satisfaction ratings, number of bedrooms, distance from the city center, and more. The dataset provides a detailed understanding of airbnb prices for both weekdays and weekends. By employing spatial econometric methods, we analyze the factors that influence airbnb prices across these cities.

The airbnb in our dataset are located in 10 cities in Europe which include Amsterdam, Athens, Barcelona, Berlin, Rome, Vienna, Paris, London, Lisbon, Budapest.

Source: https://www.kaggle.com/datasets/thedevastator/airbnb-prices-in-european-cities



**Methodology**

The first step is doing the data cleaning and exploratory data analysis. 

Next, we implemented different Machine Learning models to predict the airbnb prices:
- Decision Tree
- Random Forest
- XGBoost
- Linear Regression
- Ridge Regression
- Lasso Regression
- Neural Network

We also applied techniques such as feature selection, target variable log-transformation and hyperparameter tuning to improve model performance.

In [ ]:
# Data Loading
data_files = [
    './dataset/amsterdam_weekdays.csv',
    './dataset/amsterdam_weekends.csv',
    './dataset/athens_weekdays.csv',
    './dataset/athens_weekends.csv',
    './dataset/barcelona_weekdays.csv',
    './dataset/athens_weekends.csv',
    './dataset/barcelona_weekdays.csv',
    './dataset/barcelona_weekends.csv',
    './dataset/berlin_weekdays.csv',
    './dataset/berlin_weekends.csv',
    './dataset/budapest_weekdays.csv',
    './dataset/budapest_weekends.csv',
    './dataset/lisbon_weekdays.csv',
    './dataset/lisbon_weekends.csv',
    './dataset/london_weekdays.csv',
    './dataset/london_weekends.csv',
    './dataset/paris_weekdays.csv',
    './dataset/paris_weekends.csv',
    './dataset/rome_weekdays.csv',
    './dataset/rome_weekends.csv',
    './dataset/vienna_weekdays.csv',
    './dataset/vienna_weekends.csv'
]

In [ ]:
df = []

for index in range(len(data_files)):
    data_file = pd.read_csv(data_files[index])

    file_info = data_files[index].split('/')[2].split('_')
    
    data_file['city'] = file_info[0]
    data_file['day'] = file_info[1].split('.')[0]
    
    df.append(data_file)

df = pd.concat(df)
    

In [ ]:
df = df.drop(columns=['Unnamed: 0'])
df

In [ ]:
df = df.reset_index(drop=True)
df


## **2. Data Exploration and Cleaning**

### **2.1. Dataset summary and initial cleaning**

In [ ]:
df.info()

In [ ]:
# Check missing values
# There is no missing values 
df.isna().sum().sum()

In [ ]:
# Drop duplicates
df = df.drop_duplicates()

In [ ]:
# Over 4000 duplicated rows dropped
df = df.reset_index(drop=True)
df

In [ ]:
# Save df in a csv file
df.to_csv('./dataset/europe_airbnb.csv')

In [ ]:
df.describe()

### **2.2. Inspection on values and distributions of variables**

In [ ]:
df.columns

In [ ]:
# visually inspect the data - numerical columns
df.hist(bins=50, figsize=(15,10))

Our target variable is realSum - which is highly right skewed. Further investigation on the outliers should be conducted. 

Other data features:
- There's a long tail to a few variables such as attr_index, metro_dist, guest_satisfaction_overall
- They are at different scales

In [ ]:
# Look at the distributions of log transformed realSum
# Convert the realSum to log to reduce the effect of the long tail 
# and make the distrubtion more gaussian

fig, axs = plt.subplots(figsize=(8, 3))
sns.histplot(x=np.log1p(df['realSum']), bins=40, ax=axs).set(
    xlabel='realSum (Log)',
    title='Distribution of realSum (Log)'
)

plt.show()

In [ ]:
# Categorical columns inspection
cat_cols = [
    'room_type',
    'room_shared',
    'room_private',
    'host_is_superhost',
    'city',
    'day',
    'multi',
    'biz'
]

In [ ]:
# Distribution of categorical columns
for col in cat_cols:
    fig, ax = plt.subplots(figsize=(5, 2))
    sns.countplot(y=col, data=df).set(ylabel=None, title=f'Count of airbnb by {col}')

The plots show that:
- Most airbnbs are entire home/apt and private rooms. Only a very small number is shared room
- Most hosts are not superhosts
- The majority of our datasets are airbnbs in London, Rome, Paris and Lisbon. Those are also the capital cities with bigger sizes 
- The data points of weekdays and weekends are quite equal


### **2.3. Inspection into relationship between variables**

Feature transforms

In [ ]:
df['log_realSum'] = np.log1p(df['realSum'])

In [ ]:
# Convert data types of categorical columns to string
df[cat_cols] = df[cat_cols].astype(str)

In [ ]:
df.info()

In [ ]:
# Distributions of categorical columns by Log realSum
for col in cat_cols:
    fig, ax = plt.subplots(figsize=(8,3))
    sns.boxplot(x='log_realSum', y=col, data=df).set(
        title=f'Distribution of log price by {col}',
    )

The plots show that:
- Prices of entire home/ apt is highest, followed by private room and shared room
- If the room is private, price is higher
- Prices of airbnb in Amsterdam is higest. The 2nd expensive city is Paris, the 3rd is London
- There is no difference of prices between weekdays and weekends, whether the hosts have more than 4 offers or not, and whether the host is superhost or not.

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 12))
sns.heatmap(
    df.corr(numeric_only=True), annot=True, cmap="RdBu", annot_kws={"size": 15}, vmin=-1, vmax=1, fmt='.1f'
)
plt.show()

Overall, there are a few variables which have high correlations with others. Notably, we have:
- log_realSum & attr_index_norm: positive correlation - the higher the attraction index of the airbnb, the higher its price is
- log_realSum & persons_capacity: if more people can stay in an airbnb, its price will be higher. This can be similar with the positive correlation with log_realSum and bedrooms
- log_realSum & rest_index_norm: if the restaurant index is higher, the airbnb's price will be higher.
- attr_index & rest_index: restaurant index is high -> attraction index of the listing will also be high.

Multicollinearity exists so data pre-processing to solve this issue is required if we want to use linear regression model.

In [ ]:
# Save df in a csv file
df.to_csv('airbnb.csv')

### **2.4. Data Preparation**

In [ ]:
# One hot encoding
dfnew = pd.get_dummies(df, drop_first=True, columns=cat_cols, dtype=int)
dfnew

In [ ]:
X = dfnew.drop(columns=['realSum', 'log_realSum'], axis=1)
X

In [ ]:
y1 = dfnew['realSum']

In [ ]:
# We use the log-transformed price as the target variable
y = dfnew['log_realSum']

In [ ]:
num_cols = [
    'person_capacity',
    'cleanliness_rating',
    'guest_satisfaction_overall',
    'bedrooms',
    'dist',
    'metro_dist',
    'attr_index',
    'attr_index_norm',
    'rest_index',
    'rest_index_norm',
    'lng',
    'lat'
]

In [ ]:
# Scaling data
scale = StandardScaler()
X_num_scaled = scale.fit_transform(X[num_cols])

X_num_scaled = pd.DataFrame(X_num_scaled, columns=num_cols,
                        index=X.index)

X_num_scaled

In [ ]:
X_cat_cols = X.drop(columns = num_cols)
X_cat_cols

In [ ]:
X_scaled = pd.concat([X_cat_cols, X_num_scaled], axis=1)
X_scaled

In [ ]:
df_scaled = pd.concat([X_scaled, y], axis=1)
df_scaled.to_csv('prep_data.csv')

In [ ]:
df_scaled = pd.concat([X_scaled, y, y1], axis=1)
df_scaled.to_csv('prep_data_ver2.csv', index=False)

## **3. Modelling**

In [ ]:
def compute_metrics(y, y_pred):
    return {
      'MAE': mean_absolute_error(y, y_pred),
      'MAPE': mean_absolute_percentage_error(y, y_pred) * 100,
      'R2': r2_score(y, y_pred),
      'MSE': mean_squared_error(y, y_pred)
    }

**We will use train/test split with a size of 70/30 in all models implementation.**

### **3.1. Baseline - Mean Prediction**

The first model is a baseline which only predicts the average premium, regardless of the input variables

In [ ]:
# Train Test Split
X_train_mean, X_test_mean, y_train_mean, y_test_mean = train_test_split(
    X_scaled, y, test_size=0.3, random_state=123
    )

In [ ]:
predicted_mean = y_train_mean.mean()

y_pred_train_mean = np.repeat(predicted_mean, len(y_train_mean))
y_pred_test_mean = np.repeat(predicted_mean, len(y_test_mean))

print('Train: ', compute_metrics(np.expm1(y_train_mean), np.expm1(y_pred_train_mean)))
print('Test: ', compute_metrics(np.expm1(y_test_mean), np.expm1(y_pred_test_mean)))

Based on the model performance above, we can see that this baseline is not reliable.
The R2 is negative and the MAPE is very high at around 0.49.

Hence, more advanced machine learning models should be implemented for this task.

### **3.2 Decision Tree**

Based on the characteristics of tree based algorithms, we decided that:
- We dropped the columns 'attr_index_norm', 'rest_index_norm' since Decision Tree doesn't require data normalisation and we already have the original values
- We dropped 'log_realSum' and used 'realSum' as the decision tree model is no sensitive to outliers

Source:
https://www.sciencedirect.com/topics/mathematics/decision-tree

In [ ]:
# Data pre-processing
df_dt = dfnew.drop(columns = ['attr_index_norm', 'rest_index_norm', 'log_realSum'], axis = 1)
df_dt

In [ ]:
# Identify the X and Y
dt_X = df_dt.drop(columns=["realSum"], axis = 1)
dt_Y = df_dt["realSum"]

In [ ]:
# Separate the dataset into training/testing with percentage 70%/30%
dt_x_train, dt_x_test, dt_y_train, dt_y_test = train_test_split(dt_X, dt_Y, test_size=0.3, random_state=123)

print(f"Training on {len(dt_x_train)} observations", "\n"
      f"Testing on {len(dt_x_test)} observations")

In [ ]:
# Set up the decision tree model
dt_model = DecisionTreeRegressor(random_state = 50)

# Set the hyper-parameters of the model
parameters = {
    'max_depth':[3, 5, 8, 10],
    'min_samples_split': [2, 3, 5, 8],
    'min_samples_leaf': [1, 5, 8, 10],
    'splitter':['best','random']
}

# Perform grid search with cross validation = 5
grid_search = GridSearchCV(dt_model, parameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
# Train the model and find the best hyper-parameters
grid_search.fit(dt_x_train, dt_y_train)

In [ ]:
# Identify the best model with the best hyper-parameters
best_parameters = grid_search.best_params_
print("Best Parameters:", best_parameters)
best_model_dt = grid_search.best_estimator_
print("Best Model:", best_model_dt)

In [ ]:
# Predict the test data and calculate the evaluation metrics
dt_y_pred = best_model_dt.predict(dt_x_test)
print("Decision Tree" "\n", compute_metrics(dt_y_test, dt_y_pred))

In [ ]:
# Visualise the tree
dot_data = tree.export_graphviz(best_model_dt, out_file=None,
                                feature_names= dt_X.columns,
                                filled=True, rounded=True,
                                special_characters=True)
display(graphviz.Source(dot_data))

In [ ]:
# Feature importance
feature_importance = pd.Series(
    best_model_dt.feature_importances_,
    index=best_model_dt.feature_names_in_,
).sort_values()

# Filter feature importance > 0
feature_importance = feature_importance[feature_importance > 0]

# Plot the feature importance
fig, ax = plt.subplots(figsize=(5, 8), dpi=500)
bars = ax.barh(feature_importance.index, feature_importance)
ax.bar_label(bars)
for bars in ax.containers:
    ax.bar_label(bars)

# Add labels and title
plt.ylabel('Feature')
plt.xlabel('Weight')
plt.title('Feature importance of the Tuned Decision Tree Model')
plt.show()

The figure above illustrates the significance of various features in determining Airbnb prices in Europe. The most crucial feature, denoted as "attr_index," holds an important value of 0.357111. Following closely is the feature "bedrooms" with an importance value of 0.16935, and "lat" ranks third with a value of 0.154011. These three features account for more than 65% of the total feature importance in the decision tree model. Therefore, they play a pivotal role in influencing Airbnb prices in Europe.

Comparing the performance of the decision tree to the baseline model, both R2 values are negative and the baseline model and decision tree model are -0.02588 and -0.21567 respectively. This indicates that neither of the models is performing well in fitting the data. Therefore, it is advisable to consider implementing other machine learning models.


### **3.3 Random Forest**

We used the same set with the decision tree model. Random Forest algorithm exhibits reduced sensitivity to outliers and noisy data compared to other algorithms due to its ensemble nature, which involves averaging predictions from multiple decision trees. Hence, the target variable is the orginial price, not the log-transformed price.

In [ ]:
# Identify the X and Y
rf_X = df_dt.drop(columns=["realSum"], axis = 1)
rf_Y = df_dt["realSum"]

In [ ]:
# Separate the dataset into training/testing with percentage 70%/30%
rf_x_train, rf_x_test, rf_y_train, rf_y_test = train_test_split(rf_X, rf_Y, test_size=0.3, random_state=123)

print(f"Training on {len(rf_x_train)} observations", "\n"
      f"Testing on {len(rf_x_test)} observations")

In [ ]:
# Set up the randome forest model
rf_model = RandomForestRegressor(random_state = 889)

# Set the hyper-parameters of the model
parameters = {
    'n_estimators': [50, 200, 300], 
    'max_features': [2, 4, 6, 8],
    'max_depth': [4, 6, 8],
    'min_samples_leaf': [0.1, 0.2]
}
# Perform grid search with cross validation = 5
rand_search = RandomizedSearchCV(rf_model, 
                                parameters,
                                cv=5, 
                                n_jobs=-1,
                                scoring='neg_mean_squared_error', 
                                n_iter=10)

In [ ]:
# Train the model and find the best hyper-parameters
rand_search.fit(rf_x_train, rf_y_train)

In [ ]:
# Identify the best model with the best hyper-parameters
best_parameters = rand_search.best_params_
print("Best Parameters:", best_parameters)
best_rf = rand_search.best_estimator_
print("Best Model:", best_rf)

In [ ]:
# Predict the test data and calculate the evaluation metrics
rf_y_pred = best_rf.predict(rf_x_test)
print("Random Forest" "\n", compute_metrics(rf_y_test, rf_y_pred))

In [ ]:
# Feature importance
feature_importance = pd.Series(
    best_rf.feature_importances_,
    index=best_rf.feature_names_in_,
).sort_values()

# Plot the feature importance
fig, ax = plt.subplots(figsize=(5, 15), dpi=500)
bars = ax.barh(feature_importance.index, feature_importance)
ax.set_xlabel('Weight')
ax.set_ylabel('Feature')
ax.set_title('Feature importance of the Random Forest model')

ax.bar_label(bars)
for bars in ax.containers:
    ax.bar_label(bars)


Compared to the decision tree, the random forest enables better identification of the importance of a feature, and also it is robust to outliers. The influence of outliers is mitigated via the average effect of multiple trees. Moreover, the random forest is well-suited for modeling high-dimensional data due to its ability to handle missing values, as well as its versatility in accommodating various data types, including continuous, categorical, and binary variables. Also, the random forest is strong enough to overcome the overfitting problems without pruning the trees due to the bootstrapping and ensemble scheme.

The above figure illustrates the significance of features in the random forest model. Among the features, "lat" holds the highest importance value of 0.281221, followed by "lng" and "person_capacity" with importance values of 0.195311 and 0.129157 respectively. These three features contribute to over 60% of the total feature importance in the random forest model. Furthermore, the prominence of latitude and longitude highlights their significant impact on airbnb prices.

Comparing the performance of the random forest model to the decision tree model and baseline model, a positive R2, which is 0.13243 indicates a relatively better fit and some degree of relationship between the random forest model and Airbnb prices. However, it is worth noting that the R2 value of 0.13243 still suggests room for improvement in accurately capturing the data patterns because the range of R2 is from 0 to 1, with values closer to 1 indicating a stronger fit between the model and the data.



### **3.4 XGBoost**

### 3.4.1 XGBoost with original target variable

In [ ]:
# Identify the X and Y
xgb_prepared = df_dt.drop(columns=["realSum"], axis = 1)
xgb_labels = df_dt["realSum"]

In [ ]:
# Split the data into training and testing sets
# These are the same training and test sets as used in Decision Tree and Random Forest models
X_train, X_test, y_train, y_test = train_test_split(xgb_prepared, xgb_labels, test_size=0.3, random_state=123)

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'max_depth': [2, 3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
}

# Create the XGBoost regressor
model = XGBRegressor(objective='reg:squarederror')

# Perform grid search with cross-validation
xgb_rand_search = RandomizedSearchCV(
    model, 
    param_grid, 
    cv=5, 
    n_jobs=-1,
    scoring='neg_mean_squared_error')

xgb_rand_search.fit(X_train, y_train)


In [ ]:
# Identify the best model with the best hyper-parameters
best_parameters = xgb_rand_search.best_params_
print("Best Parameters:", best_parameters)
best_model_xgb = xgb_rand_search.best_estimator_
print("Best Model:", best_model_xgb)

In [ ]:
# Make predictions
xgb_y_pred = best_model_xgb.predict(X_test)

print("XGBoost" "\n", compute_metrics(y_test, xgb_y_pred))

In [ ]:
# Feature importance
feature_importance = pd.Series(
    best_model_xgb.feature_importances_,
    index=best_model_xgb.feature_names_in_,
).sort_values()

# Filter feature importance > 0
feature_importance = feature_importance[feature_importance > 0]

# Plot the feature importance
fig, ax = plt.subplots(figsize=(5, 8), dpi=500)
bars = ax.barh(feature_importance.index, feature_importance)
ax.bar_label(bars)
for bars in ax.containers:
    ax.bar_label(bars)

# Add labels and title
plt.ylabel('Feature')
plt.xlabel('Weight')
plt.title('Feature importance of the XGBoost Model \n with original target variable')
plt.show()

### 3.4.2 XGBoost with log-transformed target variable

The presence of outliers in the context of boosting can be detrimental due to the construction of each subsequent tree based on the residuals or errors of the previous trees. Outliers tend to exhibit substantially larger residuals compared to non-outliers, consequently causing gradient boosting to allocate a disproportionate emphasis on these data points.

Therefore, it is worth to be considered whether handling the skewness in the target variable can improve the performance of XGBoost model or not. Instead of removing the outliers, we can try to log transform the target variable.

In [ ]:
# Identify the X and Y
xgb_prepared = df_dt.drop(columns=["realSum"], axis = 1)
xgb_labels_2 = dfnew["log_realSum"]

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(xgb_prepared, xgb_labels_2, test_size=0.3, random_state=123)

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'max_depth': [2, 3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
}

# Create the XGBoost regressor
model = XGBRegressor(objective='reg:squarederror')

# Perform grid search with cross-validation
xgb_rand_search = RandomizedSearchCV(
    model, 
    param_grid, 
    cv=5, 
    n_jobs=-1,
    scoring='neg_mean_squared_error')

xgb_rand_search.fit(X_train, y_train)


In [ ]:
# Identify the best model with the best hyper-parameters
best_parameters = xgb_rand_search.best_params_
print("Best Parameters:", best_parameters)
best_model_xgb = xgb_rand_search.best_estimator_
print("Best Model:", best_model_xgb)

In [ ]:
# Make predictions
xgb_y_pred = best_model_xgb.predict(X_test)

print("XGBoost with log transformation" "\n", compute_metrics(y_test, xgb_y_pred))

In [ ]:
print('XGBoost with log transformation: \n ', compute_metrics(np.expm1(y_test), np.expm1(xgb_y_pred)))

From the result above, the MSE does not reduce significantly compared with the 1st XGBoost model's MSE. 

However, the MAE and MAPE and R2 of the 2nd XGBoost are all better, which indicates an overall better performance than the 1st XGBoost.

In [ ]:
# Feature importance
feature_importance = pd.Series(
    best_model_xgb.feature_importances_,
    index=best_model_xgb.feature_names_in_,
).sort_values()

# Filter feature importance > 0
feature_importance = feature_importance[feature_importance > 0]

# Plot the feature importance
fig, ax = plt.subplots(figsize=(5, 8), dpi=500)
bars = ax.barh(feature_importance.index, feature_importance)
ax.bar_label(bars)
for bars in ax.containers:
    ax.bar_label(bars)

# Add labels and title
plt.ylabel('Feature')
plt.xlabel('Weight')
plt.title('Feature importance of the XGBoost Model with \n log transformation of the target variable')
plt.show()

### 3.4.3 XGBoost with original target variable and feature selection

Since there are features that have high correlation with the other, feature selection is considered when running the third XGBoost. We tried this method with the original price, not the price with log transformation.

In [ ]:
# Reading datasets
df = pd.read_csv("./prep_data_ver2.csv")

In [ ]:
df

In [ ]:
# Replace logtransformed price by original price
df = df.drop(columns = ["log_realSum"], axis = 1)

In [ ]:
# Seperate features and labels
xgb_prepared = df.drop(columns=["realSum"], axis = 1)
xgb_labels = df["realSum"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(xgb_prepared, xgb_labels, test_size=0.3, random_state=42)

In [ ]:
# Create and train the XGBoost regressor
model = XGBRegressor(objective='reg:squarederror')  # Use 'reg:squarederror' for regression
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the performance using mean squared error (MSE)
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

# Get feature importance scores
importance_scores = model.feature_importances_
feature_names = X_train.columns

# Sort the feature importance scores and feature names in descending order
sorted_indices = importance_scores.argsort()[::-1]
sorted_scores = importance_scores[sorted_indices]
sorted_feature_names = [feature_names[i] for i in sorted_indices]


In [ ]:
from sklearn.metrics import mean_squared_error

mse_results = []  # Store MSE values for different values of k
k_values = range(1, len(feature_names) + 1)  # Range of k values to consider

for k in k_values:
    # Select top k features based on importance scores
    selected_features = sorted_feature_names[:k]

    # Seperate selected features
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    # Create and train the XGBoost regressor with selected features
    model_selected = XGBRegressor(objective='reg:squarederror')
    model_selected.fit(X_train_selected, y_train)

    # Make predictions using the model with selected features
    predictions_selected = model_selected.predict(X_test_selected)

    # Calculate mean squared error (MSE) with selected features
    mse_selected = mean_squared_error(y_test, predictions_selected)

    # Store the MSE value
    mse_results.append(mse_selected)

# Find the index with the minimum MSE value
optimal_k_index = mse_results.index(min(mse_results))
optimal_k = k_values[optimal_k_index]

print("MSE Results:", mse_results)
print("Optimal k:", optimal_k)


In [ ]:
# Select optimal k values based on mse
k = optimal_k  # Number of top features to select
selected_features = sorted_feature_names[:k]
print("Selected Features:", selected_features)

In [ ]:
# Refit the model based on the optimal k value
# Seperate selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Create and train the XGBoost regressor with selected features
model_selected = XGBRegressor(objective='reg:squarederror')
model_selected.fit(X_train_selected, y_train)

# Make predictions using the model with selected features
predictions_selected = model_selected.predict(X_test_selected)

# Evaluate the performance using mean squared error (MSE) with selected features
mse_selected = mean_squared_error(y_test, predictions_selected)
print("Mean Squared Error (Selected Features):", mse_selected)
# Calculate R-squared
r_squared = r2_score(y_test, predictions_selected)

print("R-squared:", r_squared)



In [ ]:
mae = mean_absolute_error(y_test, predictions_selected)
print("Mean Absolute Error:", mae)

In [ ]:
# Define the parameter grid for randomize search
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 300, 500],
}

# Create the XGBoost regressor
model = XGBRegressor(objective='reg:squarederror')

# Perform grid search with cross-validation
grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions
predictions = best_model.predict(X_test_selected)


In [ ]:
# # Evaluate the performance using mean squared error (MSE)
# mse = mean_squared_error(y_test, predictions)
# print("Mean Squared Error:", mse)

# # Print the best hyperparameters
# print("Best Hyperparameters:", grid_search.best_params_)

# # Make predictions
# xgb_y_pred = best_model.predict(X_test)

In [ ]:
# Seperate selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Create and train the XGBoost regressor with selected features
model_selected = XGBRegressor(objective='reg:squarederror')
model_selected.fit(X_train_selected, y_train)

# Make predictions using the model with selected features
predictions_selected = model_selected.predict(X_test_selected)

# Evaluate the performance using mean squared error (MSE) with selected features
mse_selected = mean_squared_error(y_test, predictions_selected)
print("Mean Squared Error (Selected Features):", mse_selected)
print("XGBoost with feature selection" "\n", compute_metrics(y_test, predictions_selected))


In [ ]:
# Visualise feature importance
importance_scores = model_selected.feature_importances_
feature_names = X_train_selected.columns

# Sort the feature importance scores and feature names in descending order
sorted_indices = importance_scores.argsort()[::-1]
sorted_scores = importance_scores[sorted_indices]
sorted_feature_names = [feature_names[i] for i in sorted_indices]

# Create a feature importance plot
plt.figure(figsize=(10, 6))
plt.bar(range(len(importance_scores)), sorted_scores)
plt.xticks(range(len(importance_scores)), sorted_feature_names, rotation='vertical')
plt.xlabel('Features')
plt.ylabel('Importance Scores')
plt.title('Feature Importance')
plt.tight_layout()  # Adjust spacing
plt.show()

####  **Model performance**
The technique of feature selection is applied based on the feature importance generated by XGBoost. An iteration is used to find the optimum number of features using Mean Squared Error (MSE) as the metric. The optimum k is 13 and the corresponding MSE is 54884.70 and the R-squared value is 0.5537.

####  **Business insights**
From a business perspective, the analysis provides valuable insights into the factors that significantly impact accommodation prices. Understanding these influential features can assist businesses in several ways.

Firstly, it allows accommodation providers to set competitive pricing strategies by considering the factors that potential guests value the most. For example, according to the chart 'Feature Importance' we can see that  accommodations with private rooms, a higher number of bedrooms and being close to attractions may command a premium price. Additionally, businesses can focus on enhancing specific attributes or amenities to increase guest satisfaction and overall value perception.

Furthermore, the analysis can help businesses identify potential market opportunities. For instance, if the 'city_barcelona' feature has a significant impact on prices, it suggests that accommodations in Barcelona may have higher market demand or specific characteristics that attract guests, enabling businesses to tailor their marketing and operational strategies accordingly.

### **3.5 Linear Regression**

In [ ]:
df = pd.read_csv("prep_data_ver2.csv")
df.head()

Split train test and validation dataset

In [ ]:
#### Set X and y dataset separately
X = df.drop(["log_realSum", "realSum"], axis = 1)
y = df["log_realSum"]
y_real = df["realSum"]
print(X.shape, y.shape, y_real.shape)

In [ ]:
#### Split train, test, validation dataset
# split train, other dataset from original X and y (RealSum)
X_train, X_other, y_train_real, y_other = train_test_split(
    X, y_real, test_size=0.3, random_state=11
    )
# split test, validation dataset from X_other and y_other
X_test, X_valid, y_test_real, y_valid_real = train_test_split(
    X_other, y_other, test_size=0.3, random_state=11
    )
# split train, other dataset from original X and y (log_RealSum)
X_train, X_other, y_train, y_other = train_test_split(
    X, y, test_size=0.3, random_state=11
    )
# split test, validation dataset from X_other and y_other
X_test, X_valid, y_test, y_valid = train_test_split(
    X_other, y_other, test_size=0.3, random_state=11
    )
# check the shape
print(X_train.shape, X_test.shape, X_valid.shape)

### 3.5.1 Use VIF (Variance Inflation Factor) to identify features that have multicollinearity problem

#### 3.5.1.1 Check the VIF values for all features and filter those to be eliminated

In [ ]:
#### Calculate VIF of each features
# create dataframe
df_vif = pd.DataFrame()
df_vif["Feature"] = X.columns
df_vif["VIF"] = [vif(X.values, i) for i in range(X.shape[1])]
print(df_vif)

In [ ]:
#### Eliminate features with high VIF values
name_remove = df_vif.loc[df_vif["VIF"] > 5, "Feature"]
name_remove

#### 3.5.1.2 Check the VIF values for the selected features

**Strategy 1: keep coordinates remove cities**

In [ ]:
#### Calculate VIF of each features
# In strategy 1: cities would be removed and coordinates would be kept
# create dataframe
df_vif_left = pd.DataFrame()
# create the list of feature names to be excluded - combined with the manually eliminating process based on the feature descriptions
l_exclude = ["bedrooms", "dist", "attr_index", "rest_index", "room_type_Private room", "room_type_Shared room", 
             'city_athens', 'city_barcelona', 'city_berlin', 'city_budapest', 'city_lisbon', 'city_london', 'city_paris', 'city_rome', 'city_vienna']
X_left = X.drop(l_exclude, axis = 1)

df_vif_left["Feature"] = X_left.columns
df_vif_left["VIF"] = [vif(X_left.values, i) for i in range(X_left.shape[1])]
print(df_vif_left)

**Strategy 2: keep cities remove coordinates**

In [ ]:
#### Calculate VIF of each features
# In strategy 2: coordinates would be removed and cities would be kept
# create dataframe
df_vif_left = pd.DataFrame()
# create the list of feature names to be excluded - combined with the manually eliminating process based on the feature descriptions
l_exclude = ["bedrooms", "dist", "attr_index", "rest_index", "room_type_Private room", "room_type_Shared room", 
             'lat', 'lng']
X_left = X.drop(l_exclude, axis = 1)

df_vif_left["Feature"] = X_left.columns
df_vif_left["VIF"] = [vif(X_left.values, i) for i in range(X_left.shape[1])]
print(df_vif_left)

It can be concluded that both removing cities removing coordinates help reduce feature colinearity.

### 3.5.2 Implement Linear regression models
For the basic models, they don't perform cross validations so only the train and test datasets are used.

#### 3.5.2.1 Linear Regression model with all features

In [ ]:
#### Check dependent variables
print(y_test, y_test_real)

In [ ]:
#### Fit the basic linear model with log-transformation
linear_m = LinearRegression()
linear_m.fit(X_train, y_train)
y_pred_linear = linear_m.predict(X_test)

#### Fit the basic linear model without log-transformation
linear_m_real = LinearRegression()
linear_m_real.fit(X_train, y_train_real)
y_pred_linear_real = linear_m_real.predict(X_test)

In [ ]:
print('Linear Regression with Log-transformation on y after transformed back: \n ', compute_metrics(np.expm1(y_test), np.expm1(y_pred_linear)))

In [ ]:
print('Linear Regression without Log-transformation: \n ', compute_metrics( y_test_real, y_pred_linear_real))

In [ ]:
plt.scatter(y_test, y_pred_linear)
plt.show()

In [ ]:
plt.scatter(y_test_real, np.expm1(y_pred_linear))
plt.show()

In [ ]:
plt.scatter(y_test_real, y_pred_linear_real)
plt.show()

In [ ]:
#### Show the coefficients
# Create a dataframe of coefficient names and values
coe_linear = pd.DataFrame()
coe_linear["Feature"] = linear_m.feature_names_in_
coe_linear["Value"] = linear_m.coef_
# Sort the coefficients
coe_linear = coe_linear.sort_values(by = ["Value"], ascending = False)
# Visualise the coefficients
fig, ax = plt.subplots()
ax.bar(coe_linear["Feature"], coe_linear["Value"])
ax.set_xticklabels(coe_linear["Feature"], rotation = 90, fontsize = 7)

#[i+": "+j.astype(str) for i, j in zip(linear_m.feature_names_in_, linear_m.coef_)]

The first linear model is trained on all features, and there are two features that have much higher coefficients than others. It should also be noted that model 1 without log- transformation on response variables (MAE 95.6, MAPE 39.3, MSE 46148.3, R-square 0.351) performed worse than the one on the log-transformed data (MAE 75.6, MAPE 25.7, MSE 43048.8, R- square 0.395). This result is aligned with the insight concluded from the previous visualisations. 

We decided that the group of linear regression models would all be trained and evaluated on the log-transformed response variables.

#### 3.5.2.2 Linear regression model without collinearity data.
According to the correlation matrix implemented at the data preprocessing stage, it can be inferred that many numerical features have high correlations with the other.

To reduce the model bias, features that may have high collinearities to other features are eliminated.

**Strategy 1: keep coordinates remove cities**

In [ ]:
#### Fit the basic linear model after removing the features that have collinearity or are hard to interpret
# according to the correlation matrix, the following features will be removed:
# In strategy 1: cities would be removed and coordinates would be kept
l_remove = ["bedrooms", "dist", "attr_index", "rest_index", "room_type_Private room", "room_type_Shared room", 
             'city_athens', 'city_barcelona', 'city_berlin', 'city_budapest', 'city_lisbon', 'city_london', 'city_paris', 'city_rome', 'city_vienna']
X_train_remove = X_train.drop(columns = l_remove, axis = 1)
X_test_remove = X_test.drop(columns = l_remove, axis = 1)
X_train_remove.shape

In [ ]:
#### Fit the basic linear model
linear_m_remove = LinearRegression()
linear_m_remove.fit(X_train_remove, y_train)
y_pred_linear_remove = linear_m_remove.predict(X_test_remove)

In [ ]:
print('Linear Regression with Log-transformation on coordinates: \n ', compute_metrics(np.expm1(y_test), np.expm1(y_pred_linear_remove)))

In [ ]:
#### Show the coefficients
coe_linear = pd.DataFrame()
coe_linear["Feature"] = linear_m_remove.feature_names_in_
coe_linear["Value"] = linear_m_remove.coef_
# Sort the coefficients
coe_linear = coe_linear.sort_values(by = ["Value"], ascending = False)
# Visualise the coefficients
fig, ax = plt.subplots()
ax.bar(coe_linear["Feature"], coe_linear["Value"])
ax.set_xticklabels(coe_linear["Feature"], rotation = 90, fontsize = 7)

#[i+": "+j.astype(str) for i, j in zip(linear_m_remove.feature_names_in_, linear_m_remove.coef_)]

**Strategy 2: keep cities remove coordinates**

In [ ]:
#### Fit the basic linear model after removing the features that have collinearity or are hard to interpret
# according to the correlation matrix, the following features will be removed:
# In strategy 2: coordinates would be removed and cities would be kept
l_remove = ["bedrooms", "dist", "attr_index", "rest_index", "room_type_Private room", "room_type_Shared room", 
             'lat', 'lng']
X_train_remove = X_train.drop(columns = l_remove, axis = 1)
X_test_remove = X_test.drop(columns = l_remove, axis = 1)
X_train_remove.shape

In [ ]:
#### Fit the basic linear model
linear_m_remove = LinearRegression()
linear_m_remove.fit(X_train_remove, y_train)
y_pred_linear_remove = linear_m_remove.predict(X_test_remove)

In [ ]:
print('Linear Regression with Log-transformation on cities: \n ', compute_metrics(np.expm1(y_test), np.expm1(y_pred_linear_remove)))

In [ ]:
#### Show the coefficients
coe_linear = pd.DataFrame()
coe_linear["Feature"] = linear_m_remove.feature_names_in_
coe_linear["Value"] = linear_m_remove.coef_
# Sort the coefficients
coe_linear = coe_linear.sort_values(by = ["Value"], ascending = False)
# Visualise the coefficients
fig, ax = plt.subplots()
ax.bar(coe_linear["Feature"], coe_linear["Value"])
ax.set_xticklabels(coe_linear["Feature"], rotation = 90, fontsize = 7)

#[i+": "+j.astype(str) for i, j in zip(linear_m_remove.feature_names_in_, linear_m_remove.coef_)]

Both strategies work for the linear regression model, the coefficients are making sense. MAE, MSE and adjusted R-squared score are better in the 2nd model which keeps cities and removes the coordinates (lat, lng)

Combined with the correlation matrix from the EDA, the model result indicates that some features may have multicollinearity in the dataset. To further explore the relationship between features, VIF is introduced to identify correlated features.

In the feature selection process, cities and coordinates are selected as those with high VIF values, and one of them must be excluded from the dataset. Both excluding strategies are implemented, and the model that was trained on data with cities (MAE 78.4, MAPE 26.5, MSE 45218.4, R-square 0.364) performs better than the one with coordinates (MAE 89.3, MAPE 30.6, MSE 49558.2, R-square 0.303). The coefficients are also easier to interpret compared to the first linear model. Eventually, the one trained on cities is kept as the second linear model. Comparing to the first model, the second model seems to have reasonable coefficients, at the expense of a slightly lower model performance.

### **Regularization**


Aside from eliminating the correlated features, adding a regularization term will also help reduce model bias.

Ridge regression and Lasso regression were applied in the next parts.

### 3.5.3 Ridge Regression

In [ ]:
#### Set the range of Grid Search
param_grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

In [ ]:
#### Fit the linear model with Ridge regularisation
ridge_m = Ridge()
# 5-fold cross-validation
ridge_m = GridSearchCV(ridge_m, param_grid, cv=5, n_jobs=-1)
ridge_m.fit(X_train, y_train)
# check the value of alpha
best_alpha_ridge = ridge_m.best_params_['alpha']
print(f"Best alpha for Ridge Regression: {best_alpha_ridge}")
# predict the test result
y_pred_ridge = ridge_m.predict(X_test)

In [ ]:
print('Ridge Regression with Log-transformation: \n ', compute_metrics(np.expm1(y_test), np.expm1(y_pred_ridge)))

In [ ]:
#### Show the coefficients
coe_linear = pd.DataFrame()
coe_linear["Feature"] = ridge_m.best_estimator_.feature_names_in_
coe_linear["Value"] = ridge_m.best_estimator_.coef_
# Sort the coefficients
coe_linear = coe_linear.sort_values(by = ["Value"], ascending = False)
# Visualise the coefficients
fig, ax = plt.subplots()
ax.bar(coe_linear["Feature"], coe_linear["Value"])
ax.set_xticklabels(coe_linear["Feature"], rotation = 90, fontsize = 7)

#[i+": "+j.astype(str) for i, j in zip(ridge_m.best_estimator_.feature_names_in_, ridge_m.best_estimator_.coef_)]

### 3.5.4 Lasso Regression

In [ ]:
#### Fit the linear model with Lasso regularisation
lasso_m = Lasso()
# 5-fold cross-validation
lasso_m = GridSearchCV(lasso_m, param_grid, cv=5, n_jobs=-1)
lasso_m.fit(X_train, y_train)
# check the value of alpha
best_alpha_ridge = ridge_m.best_params_['alpha']
print(f"Best alpha for Ridge Regression: {best_alpha_ridge}")
# predict the test result
y_pred_lasso = lasso_m.predict(X_test)

In [ ]:
print('Lasso Regression with Log-transformation: \n ', compute_metrics(np.expm1(y_test), np.expm1(y_pred_lasso)))

In [ ]:
#### Show the coefficients
coe_linear = pd.DataFrame()
coe_linear["Feature"] = lasso_m.best_estimator_.feature_names_in_
coe_linear["Value"] = lasso_m.best_estimator_.coef_
# Sort the coefficients
coe_linear = coe_linear.sort_values(by = ["Value"], ascending = False)
# Visualise the coefficients
fig, ax = plt.subplots()
ax.bar(coe_linear["Feature"], coe_linear["Value"])
ax.set_xticklabels(coe_linear["Feature"], rotation = 90, fontsize = 7)

#[i+": "+j.astype(str) for i, j in zip(lasso_m.best_estimator_.feature_names_in_, lasso_m.best_estimator_.coef_)]

Both regularization models are trained on all features, since these models are used for addressing the multi-collinearity problem . In addition, the Grid-Search method with cross validation was applied for hyperparameter tuning.

The two regularised models do not vary much with regard to the model performance (Ridge: MAE 75.6, MAPE 25.7, MSE 43056.1, R-square 0.395, Lasso: MAE 76.1, MAPE 25.9, MSE 43352.5, R-square 0.391), when the Ridge model slightly outperformed the Lasso model. In terms of the feature coefficients, Lasso’s coefficients vary within a smaller range.

### **3.6 MutliLayer Perceptron (MLP)**
MutliLayer Perceptron (MLP): We have tried the MultiLayer Pereceptron here to see if the neural network will handle the non-linear relationships well by learning mappings between input features and price outputs. 

In [ ]:
df_network = pd.read_csv('prep_data_ver2.csv')
# Using the log of realSum to address the skewness
X = df_network.drop(columns=['realSum', 'log_realSum'])
# we dont need the normalized index columns as we have applied our own scaling to normalize the original index columns
X.drop(columns=['attr_index_norm', 'rest_index_norm'], inplace=True)
y = df_network['log_realSum']

In [ ]:
# Split the dataset into train-val-test (40-30-30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=123)

print("Training shape: {}".format(X_train.shape))
print("Validation shape: {}".format(X_val.shape))
print("Testing shape: {}".format(X_test.shape))

In [ ]:
X_train.columns

In [ ]:
# Define the model
def train_model(hp):    
    num_units_l1 = hp.Int('num_units_l1', min_value = 200, max_value=260) 
    num_units_l2 = hp.Int('num_units_l2', min_value = 150, max_value=200) 
    num_units_l3 = hp.Int('num_units_l3', min_value = 30, max_value=50) 
    num_units_l4 = hp.Int('num_units_l4', min_value = 10, max_value=20) 
    # num_units_l5 = hp.Int('num_units_l5', min_value = 3, max_value=5) 


    dropout_rate = hp.Float('dropout_rate', min_value = 0.1, max_value=0.3) 
    learning_rate = hp.Float('learning_rate', min_value = 0.001, max_value=0.1, sampling='log') 
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(num_units_l1, activation="relu", input_shape=[X_train.shape[1]]),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(num_units_l2, activation="relu"),
        tf.keras.layers.Dense(num_units_l3, activation="relu"),
        tf.keras.layers.Dense(num_units_l4, activation="relu"),
        # tf.keras.layers.Dense(num_units_l5, activation="relu"),
        tf.keras.layers.Dense(1)])
    
    model.compile(  optimizer=tf.keras.optimizers.experimental.Adam(learning_rate = learning_rate),
                    loss='mean_squared_error',
                    metrics = ["mean_squared_error", "mean_absolute_error", "mean_absolute_percentage_error"])
    
    return model

In [ ]:
import datetime
current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the tuner
tuner = kt.Hyperband(train_model,
                     objective='val_mean_squared_error',
                     max_epochs=15,
                     factor=3,
                     directory='logs',
                     project_name='aml_' + current_timestamp)
# Perform the hypertuning
tuner.search(X_train, y_train, validation_data=(X_val,y_val))

In [ ]:
# Get the best hyperparameters from the tuner
best_hps = tuner.get_best_hyperparameters()[0]
print("Best number of hidden units:", best_hps['num_units_l1'])
print("Best number of hidden units:", best_hps['num_units_l2'])
print("Best number of hidden units:", best_hps['num_units_l3'])
print("Best number of hidden units:", best_hps['num_units_l4'])
# print("Best number of hidden units:", best_hps['num_units_l5'])
print("Best dropout rate:", best_hps['dropout_rate'])
print("Best learning rate:", best_hps['learning_rate'])

In [ ]:
# Train the model
best_model = tuner.hypermodel.build(best_hps)

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
best_model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=early_stopping_cb)

In [ ]:
# Evaluate the model
net_y_pred = best_model.predict(X_test)

# Calculate performance metrics
print('Network with log transformation: \n ', compute_metrics(np.expm1(y_test), np.expm1(net_y_pred.reshape(-1))))


In [ ]:
# Extract the encoded features from the best autoencoder model
encoder = tf.keras.Sequential(best_model_ae.layers[:4])
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)
X_test_encoded = encoder.predict(X_test)

# Define the MLP model for price prediction
def train_model(hp):    
    num_units_l1 = hp.Int('num_units_l1', min_value = 10, max_value=15) 
    num_units_l2 = hp.Int('num_units_l2', min_value = 5, max_value=10)  
    dropout_rate = hp.Float('dropout_rate', min_value = 0.1, max_value=0.3) 
    learning_rate = hp.Float('learning_rate', min_value = 0.001, max_value=0.1, sampling='log') 
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(num_units_l1, activation="relu", input_shape=[X_train_encoded.shape[1]]),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(num_units_l2, activation="relu"),
        tf.keras.layers.Dense(1)])
    
    model.compile(  optimizer=tf.keras.optimizers.experimental.Adam(learning_rate = learning_rate),
                    loss='mean_squared_error',
                    metrics = ["mean_squared_error", "mean_absolute_error", "mean_absolute_percentage_error"])
    return model

current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the tuner
tuner_mlp = kt.Hyperband(train_model,
                     objective='val_mean_squared_error',
                     max_epochs=5,
                     factor=3,
                     directory='logs',
                     project_name='aml_' + current_timestamp)
# Perform the hypertuning
tuner_mlp.search(X_train_encoded, y_train, validation_data=(X_val_encoded, y_val))

### **3.7 Autoencoder**

Here, we tried another neural network 'Autoencoder' relevant to our dataset to see if this model would give us better performance for price prediction in terms of lesser MSE and a better R2. We will train the Autoencoder and use the encoder part to extract the latent representation of new input data (dimension reduction). Next, we feed this to the MLP model for price prediction and see if this approach works better.

In [ ]:
# Define the model
def train_model(hp):    
    num_units_l1 = hp.Int('num_units_l1', min_value = 20, max_value=25) 
    num_units_l2 = hp.Int('num_units_l2', min_value = 15, max_value=20) 
    num_units_l3 = hp.Int('num_units_l3', min_value = 5, max_value=15) 

    dropout_rate = hp.Float('dropout_rate', min_value = 0.1, max_value=0.3) 
    learning_rate = hp.Float('learning_rate', min_value = 0.001, max_value=0.1, sampling='log') 
    
    # Encoder
    encoder = tf.keras.models.Sequential([ 
        tf.keras.layers.Dense(num_units_l1, activation="relu", input_shape=[X_train.shape[1]]),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(num_units_l2, activation="relu"),
        tf.keras.layers.Dense(num_units_l3, activation="relu"),
    ])

    # Decoder
    decoder = tf.keras.models.Sequential([
        tf.keras.layers.Dense(num_units_l2, activation="relu"),
        tf.keras.layers.Dense(num_units_l1, activation="relu"),
        tf.keras.layers.Dense(X_train.shape[1])
    ])

    model = tf.keras.models.Sequential([encoder, decoder])
    
    model.compile(  optimizer=tf.keras.optimizers.experimental.Adam(learning_rate = learning_rate),
                    loss='mean_squared_error',
                    metrics = ["mean_squared_error", "mean_absolute_error", "mean_absolute_percentage_error"])
    
    return model

In [ ]:
import datetime
current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the tuner
tuner_ae = kt.Hyperband(train_model,
                     objective='val_mean_squared_error',
                     max_epochs=5,
                     factor=3,
                     directory='logs',
                     project_name='aml_' + current_timestamp)
# Perform the hypertuning
tuner_ae.search(X_train, X_train, validation_data=(X_val, X_val))

In [ ]:
# Get the best hyperparameters from the tuner
best_hps_ae = tuner_ae.get_best_hyperparameters()[0]
print("Best number of hidden units:", best_hps_ae['num_units_l1'])
print("Best number of hidden units:", best_hps_ae['num_units_l2'])
print("Best number of hidden units:", best_hps_ae['num_units_l3'])
print("Best dropout rate:", best_hps_ae['dropout_rate'])
print("Best learning rate:", best_hps_ae['learning_rate'])

In [ ]:
# Train the autoencoder with the best hyperparameters
best_model_ae = tuner_ae.hypermodel.build(best_hps_ae)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
best_model_ae.fit(X_train, X_train, epochs=30, validation_data=(X_val, X_val), callbacks=early_stopping_cb)

In [ ]:
# Extract the encoded features from the best autoencoder model
encoder = tf.keras.Sequential(best_model_ae.layers[:4])
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)
X_test_encoded = encoder.predict(X_test)

# Define the MLP model for price prediction
def train_model(hp):    
    num_units_l1 = hp.Int('num_units_l1', min_value = 10, max_value=15) 
    num_units_l2 = hp.Int('num_units_l2', min_value = 5, max_value=10)  
    dropout_rate = hp.Float('dropout_rate', min_value = 0.1, max_value=0.3) 
    learning_rate = hp.Float('learning_rate', min_value = 0.001, max_value=0.1, sampling='log') 
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(num_units_l1, activation="relu", input_shape=[X_train_encoded.shape[1]]),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(num_units_l2, activation="relu"),
        tf.keras.layers.Dense(1)])
    
    model.compile(  optimizer=tf.keras.optimizers.experimental.Adam(learning_rate = learning_rate),
                    loss='mean_squared_error',
                    metrics = ["mean_squared_error", "mean_absolute_error", "mean_absolute_percentage_error"])
    return model

current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the tuner
tuner_mlp = kt.Hyperband(train_model,
                     objective='val_mean_squared_error',
                     max_epochs=5,
                     factor=3,
                     directory='logs',
                     project_name='aml_' + current_timestamp)
# Perform the hypertuning
tuner_mlp.search(X_train_encoded, y_train, validation_data=(X_val_encoded, y_val))

In [ ]:
# Get the best hyperparameters
best_hps_mlp = tuner_mlp.get_best_hyperparameters()[0]
print("Best number of hidden units:", best_hps_mlp['num_units_l1'])
print("Best number of hidden units:", best_hps_mlp['num_units_l2'])
print("Best dropout rate:", best_hps_mlp['dropout_rate'])
print("Best learning rate:", best_hps_mlp['learning_rate'])

In [ ]:
# Train the MLP 
best_model_mlp = tuner_mlp.hypermodel.build(best_hps_mlp)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
best_model_mlp.fit(X_train_encoded, y_train, epochs=30, validation_data=(X_val_encoded, y_val), callbacks=early_stopping_cb)

In [ ]:
# Evaluate the model
net2_y_pred = best_model_mlp.predict(X_test_encoded)

# Calculate performance metrics
print('Network with log transformation: \n ', compute_metrics(np.expm1(y_test), np.expm1(net2_y_pred.reshape(-1))))

In order to predict prices without explicit engineering of features, we employed a deep neural network in the form of a Multi-Layer Perceptron (MLP). The MLP was trained using supervised learning with its inputs being the independent variables of the dataset and its expected output being the price of the listing. We use MSE as the loss function for training. Hyperparameters of the network such as the number of neurons in each layer, learning rate and dropout are computed through hyperparameter tuning. With the MLP we aim to implicitly capture the complex relationship between the dependent and the independent variable. The model did not perform well, which could be due to issues with the data/features or the presence of outliers. As there is a lack of direct interpretability of the importance of individual features, it becomes difficult to spot the problem. 


In another approach, we tried an Autoencoder to compress the input data into a lower dimensional latent space representation. This representation can capture the essential features and patterns relevant to price prediction. The Autoencoder was trained with the input being the independent variables in order to reconstruct the same. Once the training is done, only the encoder is used to extract the compressed features from the dataset. These features are then input to another MLP which predicts the prices. However, this model does not outperform the MLP.


## **4. Business Insights and Applications**
This project can provide valuable insights and assistance to Airbnb owners in several ways. Firstly, by predicting Airbnb prices in European cities, owners can gain a better understanding of the factors that influence pricing and make data-driven decisions when setting their own rental rates. They can adjust their pricing strategy based on the identified significant features, such as the number of bedrooms, location coordinates, and other relevant attributes. This can help them optimize their pricing to attract more guests while ensuring competitive rates.

Furthermore, the project can help Airbnb owners identify the peak seasons and high-demand periods in specific cities. With this knowledge, owners can strategically adjust their availability and pricing during these times to maximize their occupancy rates and revenue. They can offer higher rates during peak seasons when demand is high and adjust prices accordingly during low-demand periods to maintain a steady flow of guests. This approach can lead to improved financial performance and better utilization of their rental properties.

Moreover, the insights from the project can guide Airbnb owners in enhancing their property listings. By understanding the features that have a significant impact on prices, owners can focus on improving those aspects of their accommodations. For example, they can invest in upgrading bedrooms, adding amenities, or highlighting proximity to attractions or popular landmarks. This can help them attract more bookings, increase customer satisfaction, and potentially command higher prices.

Additionally, the project's findings can assist Airbnb owners in making data-informed decisions regarding property investments. By analyzing the key factors that influence Airbnb prices, owners can evaluate potential properties for purchase or rental with a better understanding of their income-generating potential. They can identify locations and property characteristics that align with the features driving higher prices, which can guide their investment decisions and maximize their return on investment.

## **4. Conclusions and Recommendations**

Linear regression offers direct interpretability but assumes linearity and requires a high level of data pre-processing, especially in this task due to the multicollinearity issue. Meanwhile, decision trees and random forests provide intermediate interpretability and handle non-linear relationships but they are not as performant as the XGBoost models. Neural network models excel in non-linearity but can be computationally intensive and complicated to interpret results. XGBoost strikes a balance by offering strong performance, robustness to outliers, and moderate interpretability.

Among other models being used in this project, **the third XGBoost model with feature selection** stands out with its high predictive accuracy, built-in feature selection, and ability to capture non-linear relationships. It has the highest R2, and its MSE is only higher than those of linear regression models that we tried. Hence, we decided to choose this model's results as insights for strategic business analysis and implications in this case.

In addition, to enhance the insights that this analysis can bring, more information can be collected and scraped from the airbnb website and added to the dataset, such as: the number of reviews, ratings, detailed of reviews, property description (house rules, safety, cancellation policy), number of bookings per month, pricing structure, etc. These information can be helpful to analyse whether the pricing of a property is suitable or not, factors that contribute to a popular airbnb, what customers care about, etc.


